In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

In [2]:
CLIENT_ID = 'GJ3WPEWLK0BY2REIF0XDWYHZ04ENQYKZBLIWGDA0DWO4QEF5' 
CLIENT_SECRET = '33EOI05MAUUK0XQ1GKLWKHB51YXPPKNP2LE0K4Q1ERSSMUVB'
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GJ3WPEWLK0BY2REIF0XDWYHZ04ENQYKZBLIWGDA0DWO4QEF5
CLIENT_SECRET:33EOI05MAUUK0XQ1GKLWKHB51YXPPKNP2LE0K4Q1ERSSMUVB


In [3]:
address = '4190 Broadway, New York, NY'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


40.892084 -73.8971095


In [4]:
search_query = 'Restaurant'
radius = 3000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT, 'browse')
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

filtered_columns = ['name', 'categories']

dataframe_filtered = dataframe.loc[:, filtered_columns]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered = dataframe_filtered.sort_values(by=['categories'])
dataframe_filtered

,name,categories
23,Sabroso Mangu Restaurant Sporta Bar,Bar
11,Moss Café: Farm-To-Table Restaurant and Coffee...,Café
25,Nicey Nice Jamaican Restaurant,Caribbean Restaurant
43,Rochambeau Restaurant (Bill's),Caribbean Restaurant
37,Caribbean Grill Restaurant,Caribbean Restaurant
22,Yu Hua Restaurant,Chinese Restaurant
41,Kawah Chinese Restaurant,Chinese Restaurant
29,Golden Gate Chinese Restaurant,Chinese Restaurant
32,Lucky Star Chinese Restaurant,Chinese Restaurant
18,China One Chinese Restaurant,Chinese Restaurant


In [6]:
cats = {}
for index, row in dataframe_filtered.iterrows():
    if row['categories'] in cats:
        cats[row['categories']] = cats[row['categories']] + 1
    else:
        cats[row['categories']] = 1
print(cats)

{'Bar': 1, 'Café': 1, 'Caribbean Restaurant': 3, 'Chinese Restaurant': 16, 'Diner': 4, 'Food': 4, 'Indian Restaurant': 2, 'Italian Restaurant': 2, 'Japanese Restaurant': 2, 'Latin American Restaurant': 3, 'Mexican Restaurant': 5, 'Other Nightlife': 1, 'Pizza Place': 1, 'Seafood Restaurant': 2, 'Spanish Restaurant': 3}


In [7]:
address2 = '661 Manhattan Ave, Brooklyn, NY'

geolocator2 = Nominatim()
location2 = geolocator2.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print(latitude2, longitude2)

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


40.72452805 -73.9515156879646


In [8]:
search_query2 = 'Restaurant'
radius2 = 3000
print(search_query2 + ' .... OK!')

Restaurant .... OK!


In [9]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION, search_query2, radius2, LIMIT, 'browse')
results2 = requests.get(url2).json()
venues2 = results2['response']['venues']
dataframe2 = 0
dataframe2 = json_normalize(venues2)
dataframe2.head()

filtered_columns2 = ['name', 'categories']


dataframe_filtered2 = dataframe2.loc[:, filtered_columns2]
# def get_category_type(row):
#     try:
#         categories_list = row['categories']
#     except:
#         categories_list = row['venue.categories']
        
#     if len(categories_list) == 0:
#         return None
#     else:
#         return categories_list[0]['name']
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]
dataframe_filtered2 = dataframe_filtered2.sort_values(by=['categories'])
dataframe_filtered2

,name,categories
22,VIP Coffer & Restaurant,American Restaurant
26,Aunt Rosie's Restaurant,American Restaurant
44,Remedy Restaurant & Lounge,American Restaurant
1,Clinton St. Baking Co. & Restaurant,Bakery
33,Manhattan Restaurant,Café
9,Sidewalk Bar & Restaurant,Café
38,Yoli Restaurant: Comida Tipica Dominicana,Caribbean Restaurant
6,Chinese Musician Restaurant,Chinese Restaurant
13,New Apolo Restaurant,Chinese Restaurant
2,Fortune Cookies Chinese Restaurant,Chinese Restaurant


In [10]:
cats = {}
for index, row in dataframe_filtered2.iterrows():
    if row['categories'] in cats:
        cats[row['categories']] = cats[row['categories']] + 1
    else:
        cats[row['categories']] = 1
print(cats)

{'American Restaurant': 3, 'Bakery': 1, 'Café': 2, 'Caribbean Restaurant': 1, 'Chinese Restaurant': 5, 'Deli / Bodega': 1, 'Diner': 2, 'Eastern European Restaurant': 1, 'Food': 5, 'Gastropub': 1, 'Grocery Store': 1, 'Indian Restaurant': 2, 'Italian Restaurant': 5, 'Latin American Restaurant': 5, 'Lounge': 1, 'Mediterranean Restaurant': 1, 'Mexican Restaurant': 3, 'Pizza Place': 2, 'Polish Restaurant': 1, 'South American Restaurant': 1, 'Spanish Restaurant': 2, 'Sushi Restaurant': 1, 'Tapas Restaurant': 1, None: 2}
